# 📊 Análise: Relação entre Tempo de Entrega e Nota de Avaliação

## 🎯 Objetivo da Análise

Esta análise tem como objetivo investigar se existe uma **relação estatística** entre o **tempo de entrega** dos pedidos e as **notas de avaliação** dadas pelos clientes no marketplace Olist.

### 🤔 Por que esta análise é importante?

- **Experiência do Cliente**: O tempo de entrega é um dos fatores mais críticos na experiência de compra online
- **Satisfação**: Entregas mais rápidas podem resultar em clientes mais satisfeitos
- **Estratégia de Negócio**: Compreender essa relação ajuda a otimizar investimentos em logística
- **Competitividade**: Tempos de entrega são um diferencial competitivo importante

### 📋 O que vamos descobrir?

1. **Existe correlação** entre tempo de entrega e satisfação do cliente?
2. **Qual a direção** dessa correlação (positiva ou negativa)?
3. **Qual a intensidade** dessa relação?
4. **Quais insights** podemos extrair para o negócio?

---

## 🔬 Metodologia da Análise

### 📊 Abordagem Estatística

Utilizaremos a **Correlação de Pearson** para medir a relação linear entre as duas variáveis:

- **Variável X**: Tempo de entrega (em dias)
- **Variável Y**: Nota de avaliação (escala de 1 a 5)

### 🛠️ Etapas do Processo

1. **Carregamento dos Dados**: Importar datasets de pedidos e avaliações
2. **Preparação dos Dados**: Limpeza e transformação dos dados
3. **Cálculo do Tempo de Entrega**: Diferença entre datas de compra e entrega
4. **Análise Estatística**: Correlação e estatísticas descritivas
5. **Visualizações**: Gráficos para facilitar interpretação
6. **Interpretação dos Resultados**: Insights e conclusões

### 📖 Glossário de Termos

- **Correlação**: Medida estatística que indica o grau de relacionamento entre duas variáveis
- **Correlação de Pearson**: Varia de -1 a +1, onde:
  - `-1`: Correlação negativa perfeita
  - `0`: Nenhuma correlação linear
  - `+1`: Correlação positiva perfeita
- **P-valor**: Probabilidade de obter o resultado observado por acaso
- **Outliers**: Valores extremos que podem distorcer a análise

---

## 📚 Carregamento das Bibliotecas

Primeiro, vamos importar todas as bibliotecas que precisaremos para nossa análise:

In [ ]:
# Bibliotecas para manipulação de dados
import pandas as pd
import numpy as np

# Bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliotecas para análise estatística
from scipy import stats
from scipy.stats import pearsonr

# Configurações para datas
from datetime import datetime, timedelta

# Configurações para evitar warnings
import warnings
warnings.filterwarnings('ignore')

# Configurações para visualizações
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Bibliotecas carregadas com sucesso!")
print(f"📊 Versão do Pandas: {pd.__version__}")
print(f"🔢 Versão do NumPy: {np.__version__}")

## 📊 Carregamento e Preparação dos Dados

Vamos carregar os dados do Olist e preparar as tabelas necessárias para nossa análise.

In [ ]:
# Importar o módulo de carregamento de dados
from data_loader import load_data

# Carregar todos os datasets
print("🔄 Carregando dados do Olist...")
datasets = load_data()

if not datasets:
    print("❌ Erro ao carregar os dados!")
else:
    print("✅ Dados carregados com sucesso!")
    print(f"\n📋 Datasets disponíveis:")
    for name, df in datasets.items():
        print(f"  • {name}: {df.shape[0]:,} linhas x {df.shape[1]} colunas")

### 🧹 Preparação e Limpeza dos Dados

Agora vamos preparar os dados, convertendo as datas para o formato adequado e filtrando apenas os dados relevantes para nossa análise.

In [ ]:
# Extrair os datasets que precisamos
orders = datasets['orders'].copy()
reviews = datasets['order_reviews'].copy()

print("📊 Informações iniciais dos datasets:")
print(f"\n🛍️ Pedidos (orders):")
print(f"  • Total de registros: {len(orders):,}")
print(f"  • Colunas: {list(orders.columns)}")

print(f"\n⭐ Avaliações (reviews):")
print(f"  • Total de registros: {len(reviews):,}")
print(f"  • Colunas: {list(reviews.columns)}")

# Converter colunas de data para datetime
print("\n🔄 Convertendo datas para o formato adequado...")

# Datas dos pedidos
date_columns_orders = ['order_purchase_timestamp', 'order_approved_at', 
                      'order_delivered_carrier_date', 'order_delivered_customer_date', 
                      'order_estimated_delivery_date']

for col in date_columns_orders:
    if col in orders.columns:
        orders[col] = pd.to_datetime(orders[col], errors='coerce')
        print(f"  ✅ {col} convertida")

# Datas das avaliações
date_columns_reviews = ['review_creation_date', 'review_answer_timestamp']

for col in date_columns_reviews:
    if col in reviews.columns:
        reviews[col] = pd.to_datetime(reviews[col], errors='coerce')
        print(f"  ✅ {col} convertida")

print("\n✅ Preparação dos dados concluída!")

### 🔗 Combinação dos Datasets

Vamos unir (fazer um join) entre os datasets de pedidos e avaliações usando o `order_id` como chave de ligação.

In [ ]:
print("🔗 Fazendo join entre pedidos e avaliações...")

# Fazer merge entre orders e reviews usando order_id
orders_reviews = orders.merge(reviews, on='order_id', how='inner')

print(f"✅ Join realizado com sucesso!")
print(f"📊 Resultado: {len(orders_reviews):,} registros combinados")

# Mostrar informações sobre os status dos pedidos
print(f"\n📋 Status dos pedidos combinados:")
status_counts = orders_reviews['order_status'].value_counts()
for status, count in status_counts.items():
    percentage = (count / len(orders_reviews)) * 100
    print(f"  • {status}: {count:,} ({percentage:.1f}%)")

# Mostrar distribuição das notas de avaliação
print(f"\n⭐ Distribuição das notas de avaliação:")
review_counts = orders_reviews['review_score'].value_counts().sort_index()
for score, count in review_counts.items():
    percentage = (count / len(orders_reviews)) * 100
    stars = "⭐" * int(score)
    print(f"  • Nota {score} {stars}: {count:,} ({percentage:.1f}%)")

### 🎯 Filtrar Dados Relevantes

Para nossa análise, precisamos apenas dos pedidos que foram **entregues** e que possuem todas as informações necessárias (datas de compra, entrega e nota de avaliação).

In [ ]:
print("🎯 Filtrando dados relevantes para a análise...")

# Filtrar apenas pedidos entregues com dados completos
delivered_reviews = orders_reviews[
    (orders_reviews['order_status'] == 'delivered') &  # Apenas pedidos entregues
    (orders_reviews['order_delivered_customer_date'].notna()) &  # Data de entrega válida
    (orders_reviews['order_purchase_timestamp'].notna()) &  # Data de compra válida
    (orders_reviews['review_score'].notna())  # Nota de avaliação válida
].copy()

print(f"📊 Dados filtrados:")
print(f"  • Registros iniciais: {len(orders_reviews):,}")
print(f"  • Registros após filtros: {len(delivered_reviews):,}")
print(f"  • Percentual mantido: {(len(delivered_reviews) / len(orders_reviews)) * 100:.1f}%")

if len(delivered_reviews) == 0:
    print("❌ Não há dados suficientes para análise após os filtros!")
else:
    print("✅ Dados filtrados com sucesso!")
    print(f"\n📋 Primeiras 5 linhas dos dados filtrados:")
    relevant_columns = ['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date', 'review_score']
    print(delivered_reviews[relevant_columns].head())

## ⏰ Cálculo do Tempo de Entrega

Agora vamos calcular o **tempo de entrega** em dias, que é a diferença entre a data de compra e a data de entrega efetiva ao cliente.

In [ ]:
print("⏰ Calculando tempo de entrega em dias...")

# Calcular tempo de entrega em dias
delivered_reviews['tempo_entrega_dias'] = (
    delivered_reviews['order_delivered_customer_date'] - 
    delivered_reviews['order_purchase_timestamp']
).dt.days

print(f"✅ Tempo de entrega calculado!")

# Mostrar estatísticas básicas do tempo de entrega
print(f"\n📊 Estatísticas do tempo de entrega (antes da limpeza):")
tempo_stats = delivered_reviews['tempo_entrega_dias'].describe()
for stat, value in tempo_stats.items():
    print(f"  • {stat}: {value:.1f} dias")

# Verificar se há valores estranhos
tempos_negativos = (delivered_reviews['tempo_entrega_dias'] < 0).sum()
tempos_muito_altos = (delivered_reviews['tempo_entrega_dias'] > 100).sum()

print(f"\n🔍 Verificação de qualidade dos dados:")
print(f"  • Tempos negativos: {tempos_negativos} registros")
print(f"  • Tempos > 100 dias: {tempos_muito_altos} registros")

if tempos_negativos > 0 or tempos_muito_altos > 0:
    print(f"\n⚠️ Encontrados valores atípicos que precisam ser tratados!")

### 🧹 Limpeza de Outliers

Vamos remover **outliers extremos** que podem distorcer nossa análise:
- Tempos de entrega negativos (impossíveis)
- Tempos de entrega muito altos (> 100 dias)

**Por que fazer isso?**
- Tempos negativos indicam erros nos dados
- Tempos muito altos podem ser casos excepcionais que não representam a operação normal

In [ ]:
print("🧹 Removendo outliers extremos...")

# Contar registros antes da limpeza
registros_antes = len(delivered_reviews)

# Remover outliers extremos
delivered_reviews = delivered_reviews[
    (delivered_reviews['tempo_entrega_dias'] >= 0) &  # Tempo não pode ser negativo
    (delivered_reviews['tempo_entrega_dias'] <= 100)  # Limite máximo de 100 dias
]

registros_depois = len(delivered_reviews)
removidos = registros_antes - registros_depois

print(f"📊 Resultado da limpeza:")
print(f"  • Registros antes: {registros_antes:,}")
print(f"  • Registros depois: {registros_depois:,}")
print(f"  • Registros removidos: {removidos:,}")
print(f"  • Percentual mantido: {(registros_depois / registros_antes) * 100:.1f}%")

if registros_depois == 0:
    print("❌ Não há dados válidos após remoção de outliers!")
else:
    print("\n✅ Limpeza concluída!")
    
    # Mostrar novas estatísticas após limpeza
    print(f"\n📊 Estatísticas do tempo de entrega (após limpeza):")
    tempo_stats_clean = delivered_reviews['tempo_entrega_dias'].describe()
    for stat, value in tempo_stats_clean.items():
        print(f"  • {stat}: {value:.1f} dias")

## 📈 Análise Estatística

Agora vamos realizar a análise estatística principal para entender a relação entre tempo de entrega e nota de avaliação.

In [ ]:
print("📈 Iniciando análise estatística...")

# Calcular estatísticas por nota de avaliação
stats_by_score = delivered_reviews.groupby('review_score')['tempo_entrega_dias'].agg([
    'count',    # Quantidade de pedidos
    'mean',     # Tempo médio
    'median',   # Tempo mediano
    'std',      # Desvio padrão
    'min',      # Tempo mínimo
    'max'       # Tempo máximo
]).round(2)

# Renomear colunas para português
stats_by_score.columns = ['Quantidade', 'Média', 'Mediana', 'Desvio Padrão', 'Mínimo', 'Máximo']

print(f"📊 Estatísticas por nota de avaliação:")
print(stats_by_score)

# Calcular correlação de Pearson
correlation, p_value = pearsonr(
    delivered_reviews['tempo_entrega_dias'], 
    delivered_reviews['review_score']
)

print(f"\n🔍 Análise de Correlação:")
print(f"  • Correlação de Pearson: {correlation:.4f}")
print(f"  • P-valor: {p_value:.4f}")
print(f"  • Significância estatística: {'Sim' if p_value < 0.05 else 'Não'} (α = 0.05)")

# Resumo geral
print(f"\n📋 Resumo Geral da Análise:")
print(f"  • Total de avaliações analisadas: {len(delivered_reviews):,}")
print(f"  • Tempo médio de entrega: {delivered_reviews['tempo_entrega_dias'].mean():.1f} dias")
print(f"  • Nota média de avaliação: {delivered_reviews['review_score'].mean():.1f}")
print(f"  • Coeficiente de correlação: {correlation:.4f}")

### 🎯 Interpretação da Correlação

Vamos interpretar o resultado da correlação de acordo com os padrões estatísticos:

In [ ]:
print("🎯 Interpretação da Correlação:")
print(f"\n📊 Coeficiente de correlação: {correlation:.4f}")

# Interpretação baseada na força da correlação
if abs(correlation) >= 0.7:
    intensidade = "FORTE"
elif abs(correlation) >= 0.3:
    intensidade = "MODERADA"
elif abs(correlation) >= 0.1:
    intensidade = "FRACA"
else:
    intensidade = "MUITO FRACA ou INEXISTENTE"

# Interpretação baseada na direção
if correlation > 0:
    direcao = "POSITIVA"
    explicacao_direcao = "Maior tempo de entrega tende a resultar em notas maiores"
elif correlation < 0:
    direcao = "NEGATIVA"
    explicacao_direcao = "Maior tempo de entrega tende a resultar em notas menores"
else:
    direcao = "NULA"
    explicacao_direcao = "Não há relação linear detectável"

print(f"\n📈 Classificação da Correlação:")
print(f"  • Intensidade: {intensidade}")
print(f"  • Direção: {direcao}")
print(f"  • Explicação: {explicacao_direcao}")

# Interpretação detalhada para o negócio
print(f"\n💼 Interpretação para o Negócio:")

if correlation < -0.3:
    interpretacao = """Há uma correlação negativa moderada: clientes tendem a dar notas menores 
quando o tempo de entrega é maior. Isso sugere que investir em logística mais rápida 
pode melhorar significativamente a satisfação do cliente."""
elif correlation < -0.1:
    interpretacao = """Há uma correlação negativa fraca: existe uma leve tendência de notas menores 
com maior tempo de entrega, mas outros fatores também influenciam a satisfação."""
elif correlation > 0.3:
    interpretacao = """Há uma correlação positiva moderada: surpreendentemente, clientes tendem a dar 
notas maiores quando o tempo de entrega é maior. Isso pode indicar que expectativas 
foram bem gerenciadas ou que produtos com maior tempo de entrega são de maior qualidade."""
elif correlation > 0.1:
    interpretacao = """Há uma correlação positiva fraca: existe uma leve tendência de notas maiores 
com maior tempo de entrega, mas é preciso investigar outros fatores."""
else:
    interpretacao = """Não há correlação significativa: o tempo de entrega não parece ser um fator 
determinante na satisfação do cliente. Outros fatores (qualidade do produto, preço, 
atendimento) podem ser mais importantes."""

print(f"  {interpretacao}")

# Significância estatística
print(f"\n📊 Confiabilidade do Resultado:")
if p_value < 0.001:
    confiabilidade = "MUITO ALTA (p < 0.001)"
elif p_value < 0.01:
    confiabilidade = "ALTA (p < 0.01)"
elif p_value < 0.05:
    confiabilidade = "MODERADA (p < 0.05)"
else:
    confiabilidade = "BAIXA (p ≥ 0.05) - resultado pode ser devido ao acaso"

print(f"  • Confiabilidade: {confiabilidade}")
print(f"  • P-valor: {p_value:.4f}")

## 📊 Visualizações e Gráficos

Agora vamos criar visualizações para facilitar a compreensão dos resultados da análise.

In [ ]:
# Configurar o layout dos gráficos
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('📊 Análise: Tempo de Entrega vs Nota de Avaliação', fontsize=16, fontweight='bold')

# 1. Scatter Plot - Visualização da correlação
ax1 = axes[0, 0]
scatter = ax1.scatter(delivered_reviews['tempo_entrega_dias'], delivered_reviews['review_score'], 
                     alpha=0.6, color='steelblue', s=30)
ax1.set_xlabel('Tempo de Entrega (dias)')
ax1.set_ylabel('Nota de Avaliação')
ax1.set_title(f'Relação Tempo vs Avaliação\n(Correlação: {correlation:.3f})')
ax1.grid(True, alpha=0.3)

# Adicionar linha de tendência
z = np.polyfit(delivered_reviews['tempo_entrega_dias'], delivered_reviews['review_score'], 1)
p = np.poly1d(z)
ax1.plot(delivered_reviews['tempo_entrega_dias'], p(delivered_reviews['tempo_entrega_dias']), 
         "r--", alpha=0.8, linewidth=2, label=f'Linha de Tendência')
ax1.legend()

print("📈 Gráfico 1: Scatter Plot criado")

In [ ]:
# 2. Box Plot - Distribuição do tempo por nota
ax2 = axes[0, 1]
box_data = [delivered_reviews[delivered_reviews['review_score'] == score]['tempo_entrega_dias'].values 
           for score in sorted(delivered_reviews['review_score'].unique())]
box_labels = [f'Nota {int(score)}' for score in sorted(delivered_reviews['review_score'].unique())]

bp = ax2.boxplot(box_data, labels=box_labels, patch_artist=True)
colors = ['lightcoral', 'orange', 'yellow', 'lightgreen', 'lightblue']
for patch, color in zip(bp['boxes'], colors[:len(bp['boxes'])]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax2.set_xlabel('Nota de Avaliação')
ax2.set_ylabel('Tempo de Entrega (dias)')
ax2.set_title('Distribuição do Tempo por Nota')
ax2.grid(True, alpha=0.3)

print("📦 Gráfico 2: Box Plot criado")

In [ ]:
# 3. Gráfico de Barras - Tempo médio por nota
ax3 = axes[0, 2]
avg_time_by_score = delivered_reviews.groupby('review_score')['tempo_entrega_dias'].mean()
bars = ax3.bar(avg_time_by_score.index, avg_time_by_score.values, 
               color=['#FF6B6B', '#FFA500', '#FFFF00', '#90EE90', '#87CEEB'], 
               edgecolor='black', alpha=0.8)

ax3.set_xlabel('Nota de Avaliação')
ax3.set_ylabel('Tempo Médio de Entrega (dias)')
ax3.set_title('Tempo Médio de Entrega por Nota')
ax3.grid(True, alpha=0.3, axis='y')

# Adicionar valores nas barras
for bar, value in zip(bars, avg_time_by_score.values):
    ax3.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
            f'{value:.1f}d', ha='center', va='bottom', fontweight='bold')

print("📊 Gráfico 3: Gráfico de Barras criado")

In [ ]:
# 4. Histograma - Distribuição do tempo de entrega
ax4 = axes[1, 0]
ax4.hist(delivered_reviews['tempo_entrega_dias'], bins=30, color='lightgreen', 
         alpha=0.7, edgecolor='black')
ax4.axvline(delivered_reviews['tempo_entrega_dias'].mean(), color='red', 
           linestyle='--', linewidth=2, label=f'Média: {delivered_reviews["tempo_entrega_dias"].mean():.1f}d')
ax4.axvline(delivered_reviews['tempo_entrega_dias'].median(), color='orange', 
           linestyle='--', linewidth=2, label=f'Mediana: {delivered_reviews["tempo_entrega_dias"].median():.1f}d')
ax4.set_xlabel('Tempo de Entrega (dias)')
ax4.set_ylabel('Frequência')
ax4.set_title('Distribuição do Tempo de Entrega')
ax4.legend()
ax4.grid(True, alpha=0.3)

print("📈 Gráfico 4: Histograma do Tempo criado")

In [ ]:
# 5. Histograma - Distribuição das notas
ax5 = axes[1, 1]
score_counts = delivered_reviews['review_score'].value_counts().sort_index()
bars_scores = ax5.bar(score_counts.index, score_counts.values, 
                     color=['#FF6B6B', '#FFA500', '#FFFF00', '#90EE90', '#87CEEB'], 
                     alpha=0.8, edgecolor='black')

ax5.set_xlabel('Nota de Avaliação')
ax5.set_ylabel('Quantidade de Avaliações')
ax5.set_title('Distribuição das Notas de Avaliação')
ax5.grid(True, alpha=0.3, axis='y')

# Adicionar percentuais nas barras
total_reviews = len(delivered_reviews)
for bar, count in zip(bars_scores, score_counts.values):
    percentage = (count / total_reviews) * 100
    ax5.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
            f'{count}\n({percentage:.1f}%)', ha='center', va='bottom', fontweight='bold')

print("⭐ Gráfico 5: Distribuição das Notas criado")

In [ ]:
# 6. Heatmap de Correlação
ax6 = axes[1, 2]
corr_data = delivered_reviews[['tempo_entrega_dias', 'review_score']].corr()

# Criar heatmap
im = ax6.imshow(corr_data, cmap='RdYlBu_r', aspect='auto', vmin=-1, vmax=1)

# Adicionar labels
ax6.set_xticks(range(len(corr_data.columns)))
ax6.set_yticks(range(len(corr_data.columns)))
ax6.set_xticklabels(['Tempo de Entrega', 'Nota de Avaliação'], rotation=45)
ax6.set_yticklabels(['Tempo de Entrega', 'Nota de Avaliação'])

# Adicionar valores no heatmap
for i in range(len(corr_data.columns)):
    for j in range(len(corr_data.columns)):
        text = ax6.text(j, i, f'{corr_data.iloc[i, j]:.3f}',
                       ha="center", va="center", color="black", fontweight='bold')

ax6.set_title('Matriz de Correlação')

# Adicionar colorbar
plt.colorbar(im, ax=ax6, shrink=0.8)

print("🔥 Gráfico 6: Heatmap de Correlação criado")

# Ajustar layout e mostrar
plt.tight_layout()
plt.show()

print("\n✅ Todas as visualizações foram criadas com sucesso!")

## 📊 Análise Detalhada por Nota

Vamos analisar cada nota de avaliação individualmente para entender melhor o comportamento dos clientes:

In [ ]:
print("📊 Análise Detalhada por Nota de Avaliação:")
print("=" * 60)

# Análise detalhada por nota
for score in sorted(delivered_reviews['review_score'].unique()):
    score_data = delivered_reviews[delivered_reviews['review_score'] == score]['tempo_entrega_dias']
    
    print(f"\n⭐ Nota {int(score)} - {'⭐' * int(score)}")
    print(f"  📊 Quantidade de avaliações: {len(score_data):,}")
    print(f"  📊 Percentual do total: {(len(score_data) / len(delivered_reviews)) * 100:.1f}%")
    print(f"  ⏰ Tempo médio de entrega: {score_data.mean():.1f} dias")
    print(f"  ⏰ Tempo mediano de entrega: {score_data.median():.1f} dias")
    print(f"  📈 Desvio padrão: {score_data.std():.1f} dias")
    print(f"  ⚡ Tempo mínimo: {score_data.min()} dias")
    print(f"  🐌 Tempo máximo: {score_data.max()} dias")

# Encontrar insights interessantes
print("\n" + "=" * 60)
print("🔍 INSIGHTS PRINCIPAIS:")
print("=" * 60)

# Nota com menor tempo médio
tempo_medio_por_nota = delivered_reviews.groupby('review_score')['tempo_entrega_dias'].mean()
nota_menor_tempo = tempo_medio_por_nota.idxmin()
menor_tempo = tempo_medio_por_nota.min()

print(f"⚡ Nota com menor tempo médio de entrega: {int(nota_menor_tempo)} ({menor_tempo:.1f} dias)")

# Nota com maior tempo médio
nota_maior_tempo = tempo_medio_por_nota.idxmax()
maior_tempo = tempo_medio_por_nota.max()

print(f"🐌 Nota com maior tempo médio de entrega: {int(nota_maior_tempo)} ({maior_tempo:.1f} dias)")

# Diferença entre extremos
diferenca_tempo = maior_tempo - menor_tempo
print(f"📊 Diferença entre extremos: {diferenca_tempo:.1f} dias")

# Nota mais comum
nota_mais_comum = delivered_reviews['review_score'].mode()[0]
percent_nota_comum = (delivered_reviews['review_score'] == nota_mais_comum).mean() * 100
print(f"⭐ Nota mais comum: {int(nota_mais_comum)} ({percent_nota_comum:.1f}% das avaliações)")

## 💼 Insights e Conclusões de Negócio

Com base em toda a análise realizada, vamos extrair os principais insights para o negócio:

In [ ]:
print("💼 INSIGHTS E CONCLUSÕES DE NEGÓCIO")
print("=" * 50)

# Resumo executivo
print(f"\n📋 RESUMO EXECUTIVO:")
print(f"  • Total de pedidos analisados: {len(delivered_reviews):,}")
print(f"  • Período de análise: Dados históricos do Olist")
print(f"  • Correlação encontrada: {correlation:.4f}")
print(f"  • Significância estatística: {'Sim' if p_value < 0.05 else 'Não'}")

# Principais descobertas
print(f"\n🔍 PRINCIPAIS DESCOBERTAS:")

descoberta_1 = f"A correlação entre tempo de entrega e satisfação é {intensidade.lower()}"
print(f"\n1️⃣ FORÇA DA RELAÇÃO:")
print(f"   {descoberta_1} (r = {correlation:.4f})")

print(f"\n2️⃣ DISTRIBUIÇÃO DAS AVALIAÇÕES:")
for score in sorted(delivered_reviews['review_score'].unique()):
    count = (delivered_reviews['review_score'] == score).sum()
    percent = (count / len(delivered_reviews)) * 100
    print(f"   Nota {int(score)}: {count:,} avaliações ({percent:.1f}%)")

print(f"\n3️⃣ TEMPOS DE ENTREGA POR SATISFAÇÃO:")
for score in sorted(delivered_reviews['review_score'].unique()):
    tempo_medio = delivered_reviews[delivered_reviews['review_score'] == score]['tempo_entrega_dias'].mean()
    print(f"   Nota {int(score)}: {tempo_medio:.1f} dias em média")

In [ ]:
# Recomendações estratégicas
print(f"\n🎯 RECOMENDAÇÕES ESTRATÉGICAS:")

if abs(correlation) < 0.1:
    recomendacoes = [
        "FOCO EM OUTROS FATORES: O tempo de entrega não é o principal driver de satisfação. Investir em qualidade do produto, preço competitivo e atendimento ao cliente.",
        "GESTÃO DE EXPECTATIVAS: Manter comunicação clara sobre prazos de entrega para evitar frustrações.",
        "ANÁLISE MULTIFATORIAL: Investigar outros fatores que impactam a satisfação (qualidade, preço, facilidade de compra).",
        "MONITORAMENTO CONTÍNUO: Manter o tempo de entrega dentro de padrões aceitáveis, mesmo que não seja o fator crítico."
    ]
elif correlation < -0.1:
    recomendacoes = [
        "INVESTIR EM LOGÍSTICA: Melhorar processos de entrega para reduzir tempos e aumentar satisfação.",
        "PARCERIAS ESTRATÉGICAS: Desenvolver parcerias com transportadoras mais eficientes.",
        "CENTROS DE DISTRIBUIÇÃO: Considerar descentralização para reduzir distâncias de entrega.",
        "TECNOLOGIA: Implementar sistemas de rastreamento e otimização de rotas."
    ]
else:
    recomendacoes = [
        "ANÁLISE APROFUNDADA: Resultado contra-intuitivo requer investigação adicional.",
        "SEGMENTAÇÃO: Analisar por tipo de produto, região ou valor do pedido.",
        "EXPECTATIVAS: Clientes podem estar satisfeitos quando expectativas são bem gerenciadas.",
        "QUALIDADE vs VELOCIDADE: Entregas mais lentas podem estar associadas a produtos de maior qualidade."
    ]

for i, rec in enumerate(recomendacoes, 1):
    print(f"\n{i}️⃣ {rec}")

In [ ]:
# Limitações e próximos passos
print(f"\n⚠️ LIMITAÇÕES DA ANÁLISE:")
limitacoes = [
    "DADOS SIMULADOS: Esta análise utiliza dados simulados que podem não refletir a realidade completa",
    "CORRELAÇÃO ≠ CAUSAÇÃO: Correlação não implica necessariamente em relação causal",
    "FATORES EXTERNOS: Não consideramos sazonalidade, região geográfica ou categoria de produto",
    "OUTLIERS: Removemos outliers extremos que podem conter informações relevantes",
    "VARIÁVEIS OMITIDAS: Outros fatores importantes podem não estar sendo considerados"
]

for i, limitacao in enumerate(limitacoes, 1):
    print(f"\n{i}️⃣ {limitacao}")

print(f"\n🚀 PRÓXIMOS PASSOS:")
proximos_passos = [
    "DADOS REAIS: Repetir análise com dados reais do Olist quando disponíveis",
    "ANÁLISE SEGMENTADA: Analisar por categoria de produto, região e valor do pedido",
    "ANÁLISE MULTIVARIADA: Incluir mais variáveis na análise (preço, qualidade, categoria)",
    "ANÁLISE TEMPORAL: Investigar variações sazonais e tendências ao longo do tempo",
    "TESTE A/B: Implementar testes para validar hipóteses de melhoria",
    "MODELO PREDITIVO: Desenvolver modelo para prever satisfação com base em múltiplos fatores"
]

for i, passo in enumerate(proximos_passos, 1):
    print(f"\n{i}️⃣ {passo}")

print(f"\n" + "=" * 50)
print(f"✅ ANÁLISE CONCLUÍDA COM SUCESSO!")
print(f"=" * 50)

## 📚 Glossário de Termos Técnicos

Para facilitar o entendimento, aqui estão as definições dos principais termos utilizados na análise:

### 📊 **Termos Estatísticos**

**Correlação de Pearson**
- Medida que indica se duas variáveis tendem a variar juntas
- Varia de -1 a +1
- Valores próximos a 0 indicam pouca ou nenhuma correlação

**P-valor**
- Probabilidade de obter o resultado observado por acaso
- Se p < 0.05, consideramos o resultado estatisticamente significativo
- Quanto menor, mais confiável é o resultado

**Outliers**
- Valores extremos que se distanciam muito do padrão normal
- Podem distorcer análises estatísticas
- Exemplo: tempo de entrega de 500 dias (claramente atípico)

**Mediana**
- Valor do meio quando ordenamos os dados
- Menos influenciada por outliers que a média
- Útil para entender o "valor típico"

**Desvio Padrão**
- Medida de dispersão dos dados
- Indica quão "espalhados" estão os valores
- Valores altos indicam dados mais dispersos

---

### 🏢 **Termos de Negócio**

**Tempo de Entrega**
- Período entre a compra e o recebimento do produto pelo cliente
- Calculado em dias corridos
- Fator crítico na experiência do cliente

**NPS (Net Promoter Score)**
- Métrica de satisfação baseada na pergunta: "Recomendaria para um amigo?"
- Escala de 0 a 10
- Relacionado às notas de avaliação

**SLA (Service Level Agreement)**
- Acordo de nível de serviço
- Define prazos e padrões de qualidade esperados
- Base para medir performance logística

---

### 📈 **Interpretação dos Resultados**

**Correlação Forte** (|r| > 0.7)
- Existe uma relação clara entre as variáveis
- Mudanças em uma variável preveem mudanças na outra

**Correlação Moderada** (0.3 < |r| < 0.7)
- Relação perceptível, mas outros fatores também importam
- Useful para estratégia, mas não determina tudo

**Correlação Fraca** (0.1 < |r| < 0.3)
- Relação existe, mas é sutil
- Outros fatores são provavelmente mais importantes

**Sem Correlação** (|r| < 0.1)
- Não há relação linear detectável
- As variáveis parecem independentes

---

## 🎯 Conclusão Final

Esta análise demonstrou como investigar a relação entre **tempo de entrega** e **satisfação do cliente** usando técnicas estatísticas rigorosas.

### 🔑 Principais Aprendizados:

1. **Metodologia Clara**: Seguimos um processo estruturado de análise de dados
2. **Limpeza de Dados**: Tratamos outliers e dados inconsistentes
3. **Análise Estatística**: Utilizamos correlação de Pearson para medir relações
4. **Visualizações**: Criamos gráficos para facilitar interpretação
5. **Interpretação de Negócio**: Traduzimos resultados estatísticos em insights práticos

### 📊 Resultado da Análise:

- **Correlação encontrada**: Coeficiente de correlação calculado
- **Significância estatística**: Teste de hipóteses realizado
- **Interpretação**: Insights contextualizados para o negócio
- **Recomendações**: Sugestões práticas baseadas nos resultados

### 🚀 Valor Gerado:

Esta análise fornece uma base sólida para:
- **Decisões estratégicas** sobre investimentos em logística
- **Compreensão** dos drivers de satisfação do cliente
- **Benchmarking** de performance de entregas
- **Planejamento** de melhorias operacionais

---

**💡 Lembre-se**: Esta é uma análise exemplo com dados simulados. Em um cenário real, seria importante validar os resultados com dados reais e considerar fatores adicionais que podem influenciar a satisfação do cliente.

---

**📧 Para mais análises como esta, considere explorar outras variáveis do dataset Olist e aplicar técnicas mais avançadas de análise de dados!**